<a href="https://colab.research.google.com/github/hyungchanchoi/AlgorithmTrading_pykiwoom/blob/main/kodex200_kodex_active_backtest_both_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime,timedelta   ;today = datetime.today().strftime("%Y%m%d") 
from sklearn.preprocessing import minmax_scale
from statsmodels.tsa.stattools import coint
pd.options.display.float_format = '{:.6f}'.format
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


In [ ]:
cd drive/My Drive/algorithmtrading/pair_trading

/content/drive/My Drive/algorithmtrading/pair_trading


In [ ]:
def one_day(temp,test_day):
    
    begin = np.where( np.array(temp.index) > str(test_day))[0][0]
    end = np.where( str(test_day+1) > np.array(temp.index) )[0][-1]
    
    if len(temp.iloc[begin:end])==0:
        print(test_day,'는 공휴일 입니다.')
    else:
        return temp.iloc[begin-1:end]

def get_period(temp,start,finish):
    
    begin = np.where( np.array(temp.index) > str(start))[0][0]
    end = np.where( str(finish+1) > np.array(temp.index) )[0][-1]
    
    return temp.iloc[begin-1:end]

def get_data(temps,start,finish):
    temps = get_period(temp,start,finish)
    temps['kodex200_dayret'] = np.log(temps['kodex200']/temps['kodex200'][0])
    temps['kodex_active_dayret'] = np.log(temps['kodex_active']/temps['kodex_active'][0])
    temps['dayret_spread_kodex'] = temps['kodex_active_dayret'] - temps['kodex200_dayret']
    temps['cumret_spread_kodex'] = temps['kodex_active_cumret'] - temps['kodex200_cumret']
    temps['tiger_active_dayret'] = np.log(temps['tiger_active']/temps['tiger_active'][0])       
    temps['dayret_spread_tiger'] = temps['tiger_active_dayret'] - temps['kodex200_dayret']
    temps['cumret_spread_tiger'] = temps['tiger_active_cumret'] - temps['kodex200_cumret'] 
    return temps

## - 과거 1분단위 데이터 받아오기

In [ ]:
now = '_1229'

kodex200 = pd.read_pickle('kodex200'+now)
kodex_inv = pd.read_pickle('kodex_inv'+now)
# kodex_x2 = pd.read_pickle('kodex_x2'+now)
# kodex_inv = pd.read_pickle('kodex_inversex2'+now)
kodex_active = pd.read_pickle('kodex_active'+now)
tiger_active = pd.read_pickle('tiger_active'+now)
temp =  pd.merge(left = kodex200 , right = kodex_inv, how = "inner", on = "time")
temp =  pd.merge(left = temp , right = kodex_active, how = "inner", on = "time")
temp =  pd.merge(left = temp , right = tiger_active, how = "inner", on = "time")
temp.columns = ['kodex200','kodex_inv','kodex_active','tiger_active']
temp['kodex200_cumret'] = np.log(temp['kodex200']/temp['kodex200'][0])
temp['kodex_inv_cumret'] = np.log(temp['kodex_inv']/temp['kodex_inv'][0])
temp['kodex_active_cumret'] = np.log(temp['kodex_active']/temp['kodex_active'][0])
temp['cumret_spread_kodex'] = temp['kodex_active_cumret'] - temp['kodex200_cumret']
temp['ma5'] = temp['cumret_spread_kodex'].rolling(window=5, center=False).mean()
temp['ma10'] = temp['cumret_spread_kodex'].rolling(window=10, center=False).mean()
temp['ma20'] = temp['cumret_spread_kodex'].rolling(window=20, center=False).mean()
temp['ma30'] = temp['cumret_spread_kodex'].rolling(window=30, center=False).mean()
temp['tiger_active_cumret'] = np.log(temp['tiger_active']/temp['tiger_active'][0])
temp['cumret_spread_tiger'] = temp['tiger_active_cumret'] - temp['kodex200_cumret']
temp['ma5'] = temp['cumret_spread_tiger'].rolling(window=5, center=False).mean()
temp['ma10'] = temp['cumret_spread_tiger'].rolling(window=10, center=False).mean()
temp['ma20'] = temp['cumret_spread_tiger'].rolling(window=20, center=False).mean()
temp['ma30'] = temp['cumret_spread_tiger'].rolling(window=30, center=False).mean()
temp

,kodex200,kodex_inv,kodex_active,tiger_active,kodex200_cumret,kodex_inv_cumret,kodex_active_cumret,cumret_spread_kodex,ma5,ma10,ma20,ma30,tiger_active_cumret,cumret_spread_tiger
time,,,,,,,,,,,,,,
20200929.901,31030,5485,10040.000000,10100,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,0.000000,0.000000
20200929.902,31010,5485,10020.000000,10100,-0.000645,0.000000,-0.001994,-0.001349,nan,nan,nan,nan,0.000000,0.000645
20200929.903,31035,5480,10015.000000,10100,0.000161,-0.000912,-0.002493,-0.002654,nan,nan,nan,nan,0.000000,-0.000161
20200929.904,31015,5480,10005.000000,10100,-0.000484,-0.000912,-0.003492,-0.003009,nan,nan,nan,nan,0.000000,0.000484
20200929.905,31010,5480,10000.000000,10100,-0.000645,-0.000912,-0.003992,-0.003347,0.000322,nan,nan,nan,0.000000,0.000645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201229.1506,38545,4375,12260.000000,12230,0.216872,-0.226111,0.199765,-0.017107,-0.025066,-0.024540,-0.023487,-0.023283,0.191357,-0.025515
20201229.1509,38560,4370,12270.000000,12230,0.217261,-0.227254,0.200580,-0.016681,-0.024788,-0.024618,-0.023654,-0.023363,0.191357,-0.025904
20201229.1510,38545,4375,12270.000000,12225,0.216872,-0.226111,0.200580,-0.016292,-0.025259,-0.024885,-0.023842,-0.023477,0.190948,-0.025924


## - 일 단위로 데이터 저장하기 + 당일 수익률 계산

In [ ]:
df = {}
calender = []
for date in range(20200929,20201229):
    temps = pd.DataFrame()
    temps = one_day(temp,date)
    try:
        temps['kodex200_dayret'] = np.log(temps['kodex200']/temps['kodex200'][0])
        temps['kodex_inv_dayret'] = np.log(temps['kodex_inv']/temps['kodex_inv'][0])
        temps['kodex_active_dayret'] = np.log(temps['kodex_active']/temps['kodex_active'][0])
        temps['dayret_spread_kodex'] = temps['kodex_active_dayret'] - temps['kodex200_dayret']
        temps['tiger_active_dayret'] = np.log(temps['tiger_active']/temps['tiger_active'][0])
        temps['dayret_spread_tiger'] = temps['tiger_active_dayret'] - temps['kodex200_dayret']       
        df[str(date)] = temps
        calender.append(date)
    except:
        pass

# - 알고리즘

### 알고리즘 0번

In [ ]:
#Trade using a simple strategy
def algo_0( start, finish , money):

    # date = [x for x in calender if finish >= x >= start ]
    
    df = get_data(temp,start, finish)
    
    fee_buy = (1.00015)
    fee_sell = (1-0.00015)
    
    # Simulate trading
    S1 = df['kodex_active']
    S2 = df['kodex_x2']
    S3 = df['kodex_invx2']

    # Start with no money and no positions        
    count_S1 = 0
    count_S2 = 0    
    count_S3 = 0  

    spread = df['dayret_spread_kodex']  
    
    cash = money
    
    #start trading
    for i in range(1,len(df)):        
        
        money = cash
        print(i,'번째거래')
        
        if cash < min(S1[i],S2[i],S3[i]) :
            print('금액부족')
            continue  
        
        # buy short if the z-score is > 1
        if  spread[i] > threshold:   # sell S1  buy S2

            count_S1 -= lev
            count_S2 += lev

            cash +=  S1[i] * lev * fee_sell - S2[i] * lev * fee_buy

            eval_S1 = count_S1 * S1[i] *fee_buy
            eval_S2 = count_S2 * S2[i] *fee_sell
            eval_total = eval_S1+ eval_S2 

            print( '-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i], '  /  spread :',spread[i])
            print('[sell_INFO]    kodex_active  매도금액 : ',S1[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',S2[i]*lev* fee_buy,
                    'kodex_active : ',count_S1,'주  /  kodex_x2 : ',count_S2,'주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

            

        elif  spread[i] < - threshold :   # buy S1  sell S2

            count_S1 += lev
            count_S3 += lev

            cash -=  S1[i] * lev * fee_buy + S3[i] * lev * fee_buy

            eval_S1 = count_S1 * S1[i] *fee_sell
            eval_S3 = count_S3 * S3[i] *fee_sell
            eval_total = eval_S1+ eval_S3 

            print( '-----------',' kodex_active 현재가 :',S1[i],' / kodex_x2 현재가 :',S3[i], '  /  spread :',spread[i])
            print('[buy_INFO]   kodex_active 매수금액 :',S1[i]* lev*fee_buy,'/  kodex_x2 매수금액 :',S3[i]*lev* fee_buy,
                    'kodex_active : ',count_S1,'주  /  kodex_x2 : ',count_S3,'주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
    

        # Clear positions if the z-score between -.5 and .5        
        # -0.5~0.5 사이인 경우 수익일 경우 이익 실현
        elif abs(spread[i]) < threshold :           

            if count_S1 > 0 :
                eval_S1 = count_S1 * S1[i] * fee_sell
                eval_S3 = count_S3 * S3[i] * fee_sell
                eval_total = eval_S1 + eval_S3
                    
                temp_cash = cash + eval_S1 + eval_S3 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S3 = 0 
                    print('-----------','kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S3[i])
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S1,'원',' / 총매도금액 :', eval_S3  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S3,'주' ) 
            
            if count_S1 < 0 :
                eval_S1 = count_S1 * S1[i] * fee_buy
                eval_S2 = count_S2 * S2[i] * fee_sell
                eval_total = eval_S1 + eval_S2
                    
                temp_cash = cash + eval_S1 + eval_S2 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S2 = 0 
                    print('-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i])
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S1,'원',' / 총매도금액 :', eval_S2  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S2,'주' ) 

        else:
            pass  

            
    if count_S1 > 0 :
        eval_S1 = count_S1 * S1[-1] * fee_sell
        eval_S2 = count_S2 * S2[-1] * fee_sell
        eval_S3 = count_S3 * S3[-1] * fee_sell
        cash += eval_S1 + eval_S2 + eval_S3
    elif count_S1 <0:
        eval_S1 = count_S1 * S1[-1] * fee_buy
        eval_S2 = count_S2 * S2[-1] * fee_sell
        eval_S3 = count_S3 * S3[-1] * fee_sell
        cash += eval_S1 + eval_S2 + eval_S3

    return cash  

### 알고리즘 1-1번

In [ ]:
#Trade using a simple strategy
def algo_1( start, finish , money):

    # date = [x for x in calender if finish >= x >= start ]
    
    df = get_data(temp,start, finish)
    
    fee_buy = (1.00015)
    fee_sell = (1-0.00015)
    
    # Simulate trading
    S1 = df['kodex_active']
    S2 = df['kodex200']
    S3 = df['kodex_inv']
    S4 = df['tiger_active']

    # Start with no money and no positions        
    count_S1 = 0 
    count_S2 = 0    
    count_S3 = 0  
    count_S4 = 0

    spread_kodex = df['dayret_spread_kodex']
    spread_tiger = df['dayret_spread_tiger']   
    
    cash = money
    
    #start trading
    for i in range(1,len(df)):        
        
        money = cash
        print(i,'번째거래')
        
        if cash < min(S1[i],S2[i],S3[i],S4[i]) :
            print('금액부족')
            continue  
        
        # buy short if the z-score is > 1
        if  spread_kodex[i] > threshold:   # sell S1  buy S2

            count_S1 -= lev
            count_S2 += lev

            cash +=  S1[i] * lev * fee_sell - S2[i] * lev * fee_buy

            eval_S1 = count_S1 * S1[i] *fee_buy
            eval_S2 = count_S2 * S2[i] *fee_sell
            eval_total = eval_S1+ eval_S2 

            print( '-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i], '  /  spread :',spread_kodex[i])
            print('[sell_INFO]    kodex_active  매도금액 : ',S1[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',S2[i]*lev* fee_buy,
                    'kodex_active : ',count_S1,'주  /  kodex_x2 : ',count_S2,'주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
        elif  spread_tiger[i] > threshold:   # sell S1  buy S2

            count_S4 -= lev
            count_S2 += lev

            cash +=  S4[i] * lev * fee_sell - S2[i] * lev * fee_buy

            eval_S4 = count_S4 * S4[i] *fee_buy
            eval_S2 = count_S2 * S2[i] *fee_sell
            eval_total = eval_S4+ eval_S2 

            print( '-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :',S2[i], '  /  spread :',spread_tiger[i])
            print('[sell_INFO]    tiger_active  매도금액 : ',S4[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',S2[i]*lev* fee_buy,
                    'tiger_active : ',count_S4,'주  /  kodex_x2 : ',count_S2,'주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

            

        elif  spread_kodex[i] < - threshold :   # buy S1  sell S2

            count_S1 += lev
            count_S3 += lev

            cash -=  S1[i] * lev * fee_buy + S3[i] * lev * fee_buy

            eval_S1 = count_S1 * S1[i] *fee_sell
            eval_S3 = count_S3 * S3[i] *fee_sell
            eval_total = eval_S1+ eval_S3 

            print( '-----------',' kodex_active 현재가 :',S1[i],' / kodex_inv 현재가 :',S3[i], '  /  spread :',spread_kodex[i])
            print('[buy_INFO]   kodex_active 매수금액 :',S1[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',S3[i]*lev* fee_buy,
                    'kodex_active : ',count_S1,'주  /  kodex_inv : ',count_S3,'주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

        elif  spread_tiger[i] < - threshold :   # buy S1  sell S2

            count_S4 += lev
            count_S3 += lev

            cash -=  S4[i] * lev * fee_buy + S3[i] * lev * fee_buy

            eval_S4 = count_S4 * S4[i] *fee_sell
            eval_S3 = count_S3 * S3[i] *fee_sell
            eval_total = eval_S4+ eval_S3 

            print( '-----------',' tiger_active 현재가 :',S4[i],' / kodex_inv 현재가 :',S3[i], '  /  spread :',spread_tiger[i])
            print('[buy_INFO]   tiger_active 매수금액 :',S4[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',S3[i]*lev* fee_buy,
                    'tiger_active : ',count_S4,'주  /  kodex_inv : ',count_S3,'주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
    

        # Clear positions if the z-score between -.5 and .5        
        # -0.5~0.5 사이인 경우 수익일 경우 이익 실현
        elif abs(spread_kodex[i]) < threshold :           

            if count_S1 > 0 :
                eval_S1 = count_S1 * S1[i] * fee_sell
                eval_S3 = count_S3 * S3[i] * fee_sell
                eval_total = eval_S1 + eval_S3
                    
                temp_cash = cash + eval_S1 + eval_S3 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S3 = 0 
                    print('-----------','kodex_active 현재가 :',S1[i],' /  kodex_inv 현재가 :',S3[i])
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S1,'원',' / 총매도금액 :', eval_S3  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S3,'주' ) 
            elif count_S1 < 0 :
                eval_S1 = count_S1 * S1[i] * fee_buy
                eval_S2 = count_S2 * S2[i] * fee_sell
                eval_total = eval_S1 + eval_S2
                    
                temp_cash = cash + eval_S1 + eval_S2 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S2 = 0 
                    print('-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i])
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S1,'원',' / 총매도금액 :', eval_S2  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S2,'주' ) 

        elif abs(spread_tiger[i]) < threshold :  
            if count_S4 > 0 :
                eval_S4 = count_S4 * S4[i] * fee_sell
                eval_S3 = count_S3 * S3[i] * fee_sell
                eval_total = eval_S4 + eval_S3
                    
                temp_cash = cash + eval_S4 + eval_S3 
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0 
                    count_S3 = 0 
                    print('-----------','tiger_active 현재가 :',S4[i],' /  kodex_inv 현재가 :',S3[i])
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S4,'원',' / 총매도금액 :', eval_S3  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:',count_S3,'주' ) 
            elif count_S4 < 0 :
                eval_S4 = count_S4 * S4[i] * fee_buy
                eval_S2 = count_S2 * S2[i] * fee_sell
                eval_total = eval_S4 + eval_S2
                    
                temp_cash = cash + eval_S4 + eval_S2 
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0 
                    count_S2 = 0 
                    print('-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :',S2[i])
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S4,'원',' / 총매도금액 :', eval_S2  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:',count_S2,'주' ) 
        else:
            pass  

            
    if count_S1 > 0 :
        eval_S1 = count_S1 * S1[-1] * fee_sell
        eval_S2 = count_S2 * S2[-1] * fee_buy
        cash += eval_S1 + eval_S2 
    elif count_S1 <0:
        eval_S1 = count_S1 * S1[-1] * fee_buy
        eval_S2 = count_S2 * S2[-1] * fee_sell
        cash += eval_S1 + eval_S2 
    
    if count_S4 > 0 :
        eval_S4 = count_S4 * S4[-1] * fee_sell
        eval_S3 = count_S3 * S3[-1] * fee_buy
        cash += eval_S4 + eval_S3 
    elif count_S4 <0:
        eval_S4 = count_S4 * S4[-1] * fee_buy
        eval_S3 = count_S3 * S3[-1] * fee_sell
        cash += eval_S4 + eval_S3 

    return cash  

### 알고리즘 1-2번

In [ ]:
#Trade using a simple strategy
def algo_1_2( start, finish , money):

    # date = [x for x in calender if finish >= x >= start ]
    
    df = get_data(temp,start, finish)
    
    fee_buy = (1.00015)
    fee_sell = (1-0.00015)
    
    # Simulate trading
    S1 = df['kodex_active']
    S2 = df['kodex200']
    S3 = df['kodex200']
    S4 = df['tiger_active']

    # Start with no money and no positions        
    count_S1 = 0 
    count_S2 = 0    
    count_S3 = 0  
    count_S4 = 0

    spread_kodex = df['dayret_spread_kodex']
    spread_tiger = df['dayret_spread_tiger']   
    
    cash = money
    
    #start trading
    for i in range(1,len(df)):        
        
        money = cash
        print(i,'번째거래')
        
        if cash < min(S1[i],S2[i],S3[i],S4[i]) :
            print('금액부족')
            continue  
        
        # buy short if the z-score is > 1
        if  spread_kodex[i] > threshold:   # sell S1  buy S2

            count_S1 -= lev
            count_S2 += lev

            cash +=  S1[i] * lev * fee_sell - S2[i] * lev * fee_buy

            eval_S1 = count_S1 * S1[i] *fee_buy
            eval_S2 = count_S2 * S2[i] *fee_sell
            eval_total = eval_S1+ eval_S2 

            print( '-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i], '  /  spread :',spread_kodex[i])
            print('[sell_INFO]    kodex_active  매도금액 : ',S1[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',S2[i]*lev* fee_buy,
                    'kodex_active : ',count_S1,'주  /  kodex_x2 : ',count_S2,'주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
        elif  spread_tiger[i] > threshold:   # sell S1  buy S2

            count_S4 -= lev
            count_S3 += lev

            cash +=  S4[i] * lev * fee_sell - S3[i] * lev * fee_buy

            eval_S4 = count_S4 * S4[i] *fee_buy
            eval_S3 = count_S2 * S3[i] *fee_sell
            eval_total = eval_S4+ eval_S3 

            print( '-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :',S3[i], '  /  spread :',spread_tiger[i])
            print('[sell_INFO]    tiger_active  매도금액 : ',S4[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',S3[i]*lev* fee_buy,
                    'tiger_active : ',count_S4,'주  /  kodex_x2 : ',count_S3,'주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

        
        elif  spread_kodex[i] < - threshold :   # buy S1  sell S2

            count_S1 += lev
            count_S2 -= lev

            cash +=  -S1[i] * lev * fee_buy + S2[i] * lev * fee_sell

            eval_S1 = count_S1 * S1[i] *fee_sell
            eval_S2 = count_S2 * S2[i] *fee_buy
            eval_total = eval_S1+ eval_S2

            print( '-----------',' kodex_active 현재가 :',S1[i],' / kodex_inv 현재가 :',S2[i], '  /  spread :',spread_kodex[i])
            print('[buy_INFO]   kodex_active 매수금액 :',S1[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',S2[i]*lev* fee_sell,
                    'kodex_active : ',count_S1,'주  /  kodex_inv : ',count_S2,'주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

        elif  spread_tiger[i] < - threshold :   # buy S1  sell S2

            count_S4 += lev
            count_S3 -= lev

            cash +=  -S4[i] * lev * fee_buy + S3[i] * lev * fee_sell

            eval_S4 = count_S4 * S4[i] *fee_sell
            eval_S3 = count_S3 * S3[i] *fee_buy
            eval_total = eval_S4+ eval_S3 

            print( '-----------',' tiger_active 현재가 :',S4[i],' / kodex_inv 현재가 :',S3[i], '  /  spread :',spread_tiger[i])
            print('[buy_INFO]   tiger_active 매수금액 :',S4[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',S3[i]*lev* fee_sell,
                    'tiger_active : ',count_S4,'주  /  kodex_inv : ',count_S3,'주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
    

        # Clear positions if the z-score between -.5 and .5        
        # -0.5~0.5 사이인 경우 수익일 경우 이익 실현
        elif abs(spread_kodex[i]) < threshold :           

            if count_S1 > 0 :
                eval_S1 = count_S1 * S1[i] * fee_sell
                eval_S2 = count_S2 * S2[i] * fee_buy
                eval_total = eval_S1 + eval_S2
                    
                temp_cash = cash + eval_S1 + eval_S2
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S2 = 0 
                    print('-----------','kodex_active 현재가 :',S1[i],' /  kodex_inv 현재가 :',S2[i])
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S1,'원',' / 총매도금액 :', eval_S2  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S2,'주' ) 
            elif count_S1 < 0 :
                eval_S1 = count_S1 * S1[i] * fee_buy
                eval_S2 = count_S2 * S2[i] * fee_sell
                eval_total = eval_S1 + eval_S2
                    
                temp_cash = cash + eval_S1 + eval_S2 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0 
                    count_S2 = 0 
                    print('-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :',S2[i])
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S1,'원',' / 총매도금액 :', eval_S2  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:',count_S2,'주' ) 

        elif abs(spread_tiger[i]) < threshold :  
            if count_S4 > 0 :
                eval_S4 = count_S4 * S4[i] * fee_sell
                eval_S3 = count_S3 * S3[i] * fee_buy
                eval_total = eval_S4 + eval_S3
                    
                temp_cash = cash + eval_S4 + eval_S3 
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0 
                    count_S3 = 0 
                    print('-----------','tiger_active 현재가 :',S4[i],' /  kodex_inv 현재가 :',S3[i])
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S4,'원',' / 총매도금액 :', eval_S3  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:',count_S3,'주' ) 
            elif count_S4 < 0 :
                eval_S4 = count_S4 * S4[i] * fee_buy
                eval_S3 = count_S3 * S3[i] * fee_sell
                eval_total = eval_S4 + eval_S3
                    
                temp_cash = cash + eval_S4 + eval_S3
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0 
                    count_S3 = 0 
                    print('-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :',S3[i])
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S4,'원',' / 총매도금액 :', eval_S3  ,'원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:',count_S3,'주' ) 
        else:
            pass  

            
    if count_S1 > 0 :
        eval_S1 = count_S1 * S1[-1] * fee_sell
        eval_S2 = count_S2 * S2[-1] * fee_buy
        cash += eval_S1 + eval_S2 
    elif count_S1 <0:
        eval_S1 = count_S1 * S1[-1] * fee_buy
        eval_S2 = count_S2 * S2[-1] * fee_sell
        cash += eval_S1 + eval_S2 
    
    if count_S4 > 0 :
        eval_S4 = count_S4 * S4[-1] * fee_sell
        eval_S3 = count_S3 * S3[-1] * fee_buy
        cash += eval_S4 + eval_S3 
    elif count_S4 <0:
        eval_S4 = count_S4 * S4[-1] * fee_buy
        eval_S3 = count_S3 * S3[-1] * fee_sell
        cash += eval_S4 + eval_S3 

    return cash  

### 알고리즘 2번

In [ ]:
##Trade using a simple strategy
def algo_2( start, finish , money):

    # date = [x for x in calender if finish >= x >= start ]
    
    df = get_data(temp,start, finish)
    
    fee_buy = (1.00015)
    fee_sell = (1-0.00015)
    
    # Simulate trading
    S1 = df['kodex_active']
    S4 = df['tiger_active']

    # Start with no money and no positions        
    count_S1 = 0
    count_S4 = 0

    spread_kodex = df['dayret_spread_kodex']
    spread_tiger = df['dayret_spread_tiger']   
    
    cash = money
    
    #start trading
    for i in range(1,len(df)):        
        
        money = cash
        print(i,'번째거래')
        
        if cash < min(S1[i],S4[i]) :
            print('금액부족')
            continue  
        
        # buy short if the z-score is > 1
        if  spread_kodex[i] > threshold:   # sell S1  buy S2

            count_S1 -= lev

            cash +=  S1[i] * lev * fee_sell 

            eval_S1 = count_S1 * S1[i] *fee_buy
            eval_total = eval_S1

            print( '-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :''  /  spread :',spread_kodex[i])
            print('[sell_INFO]    kodex_active  매도금액 : ',S1[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',
                    'kodex_active : ',count_S1,'주  /  kodex_x2 : ''주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
        elif  spread_tiger[i] > threshold:   # sell S1  buy S2

            count_S4 -= lev

            cash +=  S4[i] * lev * fee_sell 

            eval_S4 = count_S4 * S4[i] *fee_buy
            eval_total = eval_S4

            print( '-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :' '  /  spread :',spread_tiger[i])
            print('[sell_INFO]    tiger_active  매도금액 : ',S4[i]* lev*fee_sell,'/  kodex_x2 매수금액 :',
                    'tiger_active : ',count_S4,'주  /  kodex_x2 : ''주')
            print('[sell]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

            

        elif  spread_kodex[i] < - threshold :   # buy S1  sell S2

            count_S1 += lev

            cash -=  S1[i] * lev * fee_buy 

            eval_S1 = count_S1 * S1[i] *fee_sell
            eval_total = eval_S1

            print( '-----------',' kodex_active 현재가 :',S1[i],' / kodex_inv 현재가 :', '  /  spread :',spread_kodex[i])
            print('[buy_INFO]   kodex_active 매수금액 :',S1[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',
                    'kodex_active : ',count_S1,'주  /  kodex_inv : ''주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )

        elif  spread_tiger[i] < - threshold :   # buy S1  sell S2

            count_S4 += lev

            cash -=  S4[i] * lev * fee_buy 

            eval_S4 = count_S4 * S4[i] *fee_sell
            eval_total = eval_S4

            print( '-----------',' tiger_active 현재가 :',S1[i],' / kodex_inv 현재가 :' '  /  spread :',spread_tiger[i])
            print('[buy_INFO]   tiger_active 매수금액 :',S1[i]* lev*fee_buy,'/  kodex_inv 매수금액 :',
                    'tiger_active : ',count_S1,'주  /  kodex_inv : ''주')
            print('[buy]' ,'잔고: ',round(eval_total + cash,2),'원  ', ' / 평가금액 :',eval_total,'/ 예수금 :',cash )
    

        # Clear positions if the z-score between -.5 and .5        
        # -0.5~0.5 사이인 경우 수익일 경우 이익 실현
        elif abs(spread_kodex[i]) < threshold :           

            if count_S1 > 0 :
                eval_S1 = count_S1 * S1[i] * fee_sell
                eval_total = eval_S1 
                    
                temp_cash = cash + eval_S1  
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0
                    print('-----------','kodex_active 현재가 :',S1[i],' /  kodex_inv 현재가 :')
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S1,'원',' / 총매도금액 :''원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:','주' ) 
            elif count_S1 < 0 :
                eval_S1 = -count_S1 * S1[i] * fee_buy
                eval_total = eval_S1 
                temp_cash = cash + eval_S1 
                if temp_cash > money :
                    cash = temp_cash
                    count_S1 = 0
                    print('-----------',' kodex_active 현재가 :',S1[i],' /  kodex_x2 현재가 :')
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S1,'원',' / 총매도금액 : 원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S1,'주  /  ','active:','주' ) 

        elif abs(spread_tiger[i]) < threshold :  
            if count_S4 > 0 :
                eval_S4 = count_S4 * S4[i] * fee_sell
                eval_total = eval_S4 
                    
                temp_cash = cash + eval_S4 
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0
                    print('-----------','tiger_active 현재가 :',S4[i],' /  kodex_inv 현재가 :')
                    print('[EXIT_INFO]' , '총매도금액 :',eval_S4,'원',' / 총매도금액 :','원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:','주' ) 
            elif count_S4 < 0 :
                eval_S4 = -count_S4 * S4[i] * fee_buy
                eval_total = eval_S4 
                    
                temp_cash = cash + eval_S4 
                if temp_cash > money :
                    cash = temp_cash
                    count_S4 = 0
                    print('-----------',' tiger_active 현재가 :',S4[i],' /  kodex_x2 현재가 :')
                    print('[EXIT_INFO]' , '총매수금액 :',eval_S4,'원',' / 총매도금액 :', '원')
                    print('[EXIT]' ,'잔고: ',round(cash,2),'원  ','kodex:',count_S4,'주  /  ','active:','주' ) 
        else:
            pass  

            
    if count_S1 > 0 :
        eval_S1 = count_S1 * S1[-1] * fee_sell
        cash += eval_S1 
    elif count_S1 <0:
        eval_S1 = count_S1 * S1[-1] * fee_buy
        cash += eval_S1 
    if count_S4 > 0 :
        eval_S4 = count_S4 * S4[-1] * fee_sell
        cash += eval_S4 
    elif count_S4 <0:
        eval_S4 = count_S4 * S4[-1] * fee_buy
        cash += eval_S4 

    return cash  

# 시뮬레이션

### 임계치에 따른 레버리지 테스트

In [ ]:
lev = 1
thresholde = 0.003
algo_2(20201228,20201228,3000000)

NameError: ignored

### 지금까지 과거 데이터 기준  임계치,레버리지 테스트

In [ ]:
lev = 1
profits = {}
backtest={}
x = np.linspace(0.0003,0.003,15)
temp_calender = calender[-3:]

# for dates in temp_calender:
#     profits = {}
for threshold in x:
    for lev in range(1,5):
        ret = algo_1_2(20201229,20201229,1500000)
        profits[ret] = [threshold,lev]
maxprofit = max(profits.keys())
# backtest[dates] = maxprofit, profits[maxprofit]
print(maxprofit,profits[maxprofit])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[sell] 잔고:  1290107.49 원    / 평가금액 : 78832.13775 / 예수금 : 1211275.3562499997
43 번째거래
-----------  kodex_active 현재가 : 12270.0  /  kodex_x2 현재가 : 38560   /  spread : 0.003503723343511362
[sell_INFO]    kodex_active  매도금액 :  12268.1595 /  kodex_x2 매수금액 : 38565.78400000001 kodex_active :  -4 주  /  kodex_x2 :  4 주
[sell] 잔고:  1290107.23 원    / 평가금액 : 105129.502 / 예수금 : 1184977.7317499998
44 번째거래
-----------  kodex_active 현재가 : 12270.0  /  kodex_x2 현재가 : 38545   /  spread : 0.003892803174630737
[sell_INFO]    kodex_active  매도금액 :  12268.1595 /  kodex_x2 매수금액 : 38550.78175 kodex_active :  -5 주  /  kodex_x2 :  5 주
[sell] 잔고:  1290032.0 원    / 평가금액 : 131336.88874999998 / 예수금 : 1158695.1094999998
45 번째거래
-----------  kodex_active 현재가 : 12250.0  /  kodex_x2 현재가 : 38550   /  spread : 0.0021317713444336025
[sell_INFO]    kodex_active  매도금액 :  12248.1625 /  kodex_x2 매수금액 : 38555.7825 kodex_active :  -6 주  /  kodex_x2 :  6 주
[sell] 잔고:  1290141.77 원    / 평가금액 : 1577

In [ ]:
backtest_df = pd.DataFrame(backtest).transpose() ; backtest_df.columns = ['profit','threshold & lev']
thres = [ backtest_df['threshold & lev'].values[i][0] for i in range(len(backtest_df)) ]
lev = [ backtest_df['threshold & lev'].values[i][1] for i in range(len(backtest_df)) ]
backtest_df
# pd.DataFrame(profits).transpose()

,profit,threshold & lev
20201223,3009320.378000,"[0.003, 4]"
20201224,3005303.779000,"[0.003, 4]"
20201228,4747587.779000,"[0.003, 4]"


# 백테스트

In [ ]:
test.trade('test',1000000, 5, 20)

## 윈도우 사이즈 그리드 서치

In [ ]:
# train
# length_scores = {}
# for i in range(1,20):
#     for j in range(5,60):
#         length_scores[test.trade('train',1000000, i, j)] = (i,j)
# best_length = max(length_scores.keys())
# print ('Best window length:', length_scores[best_length])
# print (best_length, 'day window:', length_scores[best_length])

# Test
length_scores2 = {}
for i in range(1,20):
    for j in range(5,60):
        length_scores2[test.trade('test',1000000, i, j)] = (i,j)
best_length2 = max(length_scores2.keys())
print (best_length2, 'day window:', length_scores2[best_length2])

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(length_scores)
plt.plot(length_scores2)
plt.xlabel('Window length')
plt.ylabel('Score')
plt.legend(['Training', 'Test'])
plt.show()

# -------------------------------------------------------------------------------------------------------------------------------------